# 2-6. 데이터 결합
## 1. 단순 연결
(1) **pandas.concat(objs, axis=0, ignore_index=False)**
> - objs: concat을 실행할 객체의 리스트(DataFrame, Series, Panel Object)
> - axis: 0이면 열 방향, 1이면 행 방향으로 합침
> - ingore_index: True이면 기존 index를 무시하고 0부터 시작하는 정수로 재설정

In [2]:
import pandas as pd

# 열 방향 결합
HR1 = pd.DataFrame({'이름':['장화','홍련'], '부서': ['영업','회계'], '직급':['팀장','사원']})
HR2 = pd.DataFrame({'이름':['콩쥐','팥쥐'], '직급':['사원','팀장'], '부서': ['영업','인사']}) # 컬럼의 위치와 관계없이 '컬럼명' 기준으로 각 열을 재조립
pd.concat([HR1, HR2], axis=0) # 열 방향

,이름,부서,직급
0,장화,영업,팀장
1,홍련,회계,사원
0,콩쥐,영업,사원
1,팥쥐,인사,팀장


In [3]:
pd.concat([HR1, HR2], axis=0, ignore_index=True) #index 0부터 시작하는 정수로 재설정

,이름,부서,직급
0,장화,영업,팀장
1,홍련,회계,사원
2,콩쥐,영업,사원
3,팥쥐,인사,팀장


In [16]:
# 결합하는 두 데이터프레임이 다른 컬럼을 가지고 있는 경우, 각 데이터프레임의 컬럼을 모두 가지고 값이 없는 칸은 NaN으로 채워짐
HR3 = pd.DataFrame({'이름':['콩쥐','팥쥐'], '부서': ['영업','인사'], '급여':[3500,2800]})
pd.concat([HR1, HR3], axis=0, ignore_index=True)

,이름,부서,직급,급여
0,장화,영업,팀장,NaN
1,홍련,회계,사원,NaN
2,콩쥐,영업,NaN,3500.0
3,팥쥐,인사,NaN,2800.0


In [17]:
# 행 방향 결합
HR4 = pd.Series({1:2500}, name='급여')
pd.concat([HR1, HR4], axis=1)

,이름,부서,직급,급여
0,장화,영업,팀장,NaN
1,홍련,회계,사원,2500.0


In [18]:
HR5 = pd.DataFrame({'급여':[4500,3000,3500]})
pd.concat([HR1, HR5], axis=1)

,이름,부서,직급,급여
0,장화,영업,팀장,4500
1,홍련,회계,사원,3000
2,NaN,NaN,NaN,3500


## 2. 조인
(1) 조인(join, merge): merge()는 좀 더 다양한 조건을 사용할 수 있음
> - 연결한 두 데이터프레임이 동시에 가지고 있는 **특정한 컬럼의 값**을 기준으로 두 개 이상의 데이터 프레임을 결합하여 하나의 데이터프레임을 생성
> - 컬럼이 가질 수 있는 값의 집합인 **도메인**이 같다는 것 (컬럼명이 같다는 것 X)
> - 방향과 수행 방식에 따라 **inner join, outer join, right join, left join**으로 구분

> **DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)**
> - DataFrame: **결합의 기준**이 되는 데이터프레임으로 **left DataFrame**
> - other: **결합의 대상**이 되는 데이터 프레임으로 **right DataFrame**
> - on: key 컬럼의 이름(두 데이터 프레임에서 key 컬럼의 이름이 같아야 함)
> - how: {'left', 'right', 'inner', 'outer'}, 조인의 수행 방식 선택
> - lsuffix/rsuffix: **key가 아닌 열 이름이 겹칠 때** 자동으로 이름을 구분해주는 접미사 옵션
> - sort: 키값을 기준으로 정렬할지 여부

> **DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x','_y'))**
> - DataFrame: **결합의 기준**이 되는 데이터프레임으로 **left DataFrame**
> - right: **결합의 대상**이 되는 데이터 프레임으로 **right DataFrame**
> - how: {'left', 'right', 'inner', 'outer', **'cross'**}, 조인의 수행 방식 선택
> - on: key 컬럼의 이름(두 데이터 프레임에서 key 컬럼의 이름이 같은 경우)
> - left_on/right_on: 왼쪽 혹은 오른쪽 데이터프레임의 인덱스를 key로 사용할 경우
> - left_index/right_index: 왼쪽 혹은 오른쪽 데이터프레임의 **인데스를 key로 사용**하는 경우
> - sort: 키값을 기준으로 정렬할지 여부
> - suffixes: 결합한 데이터프레임에서 왼쪽/오른쪽 데이터프레임의 컬럼명에 사용할 접미사를 왼쪽-오른쪽 순서로 지정

In [21]:
# inner join(교집합)
product = pd.DataFrame({'상품코드':['G1', 'G2', 'G3', 'G4'], '상품명':['우유', '감자', '빵', '치킨']})
sale = pd.DataFrame({'주문번호':[1001,1002,1003,1002,1004], '상품코드':['G4', 'G3', 'G3', 'G1', 'G5'], '주문수량':[1,4,2,2,3]})
sale.merge(product, on='상품코드', how='inner')

,주문번호,상품코드,주문수량,상품명
0,1001,G4,1,치킨
1,1002,G3,4,빵
2,1003,G3,2,빵
3,1002,G1,2,우유


In [22]:
# outer join (합집합)
sale.merge(product, on='상품코드', how='outer', sort=True)

,주문번호,상품코드,주문수량,상품명
0,1002.0,G1,2.0,우유
1,NaN,G2,NaN,감자
2,1002.0,G3,4.0,빵
3,1003.0,G3,2.0,빵
4,1001.0,G4,1.0,치킨
5,1004.0,G5,3.0,NaN


In [24]:
# left join
sale.merge(product, on='상품코드', how='left')

,주문번호,상품코드,주문수량,상품명
0,1001,G4,1,치킨
1,1002,G3,4,빵
2,1003,G3,2,빵
3,1002,G1,2,우유
4,1004,G5,3,NaN
